# 🎨 Data Designer 101: The Basics

[Click here](https://raw.githubusercontent.com/NVIDIA-NeMo/DataDesigner/refs/heads/main/docs/notebooks/1-the-basics.ipynb) to download this notebook to your computer.
#### 📚 What you'll learn

This notebook demonstrates the basics of Data Designer by generating a simple product review dataset.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMTextColumnConfig,
    ModelConfig,
    PersonSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
    UniformSamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](/models/default-model-settings.md) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🎲 Getting started with sampler columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.

<br>

You can view available samplers using the config builder's `info` property:


In [5]:
config_builder.info.display("samplers")

─────────────────────────────────────────── NeMo Data Designer Samplers ───────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Type              ┃ Parameter                     ┃ Data Type                     ┃ Required ┃ Constraints      ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ bernoulli         │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ bernoulli_mixture │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
│                   │ dist_name                     │ string                        │    ✓     │                  │
│                   │ dist_params                   │ dict                          │    ✓     │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ binomial          │ n                             │ integer                       │    ✓     │                  │
│                   │ p                             │ number                        │    ✓     │ >= 0.0, <= 1.0   │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ category          │ values                        │ string[] | integer[] |        │    ✓     │ len > 1          │
│                   │                               │ number[]                      │          │                  │
│                   │ weights                       │ number[] | null               │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ datetime          │ start                         │ string                        │    ✓     │                  │
│                   │ end                           │ string                        │    ✓     │                  │
│                   │ unit                          │ string                        │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ gaussian          │ mean                          │ number                        │    ✓     │                  │
│                   │ stddev                        │ number                        │    ✓     │                  │
│                   │ decimal_places                │ integer | null                │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ person            │ locale                        │ string                        │          │                  │
│                   │ sex                           │ string | null                 │          │                  │
│                   │ city                          │ string | string[] | null      │          │                  │
│                   │ age_range                     │ integer[]                     │          │ len > 2, len < 2 │
│                   │ state                         │ string | string[] | null      │          │                  │
│                   │ with_synthetic_personas       │ boolean                       │          │                  │
│                   │ sample_dataset_when_available │ boolean                       │          │                  │
├───────────────────┼───────────────────────────────┼───────────────────────────────┼──────────┼──────────────────┤
│ poisson           │ mean                          │ number                        │    ✓     │                  │
├───────────────────┼───────────────────────────────┼──

Let's start designing our product review dataset by adding product category and subcategory columns.


In [6]:
config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Optionally validate that the columns are configured correctly.
config_builder.validate()

[19:00:10] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: ['product_category', 'product_subcategory', 'target_age_range']
)

Next, let's add samplers to generate data related to the customer and their review.


In [7]:
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON,
        params=PersonSamplerParams(age_range=[18, 70], locale="en_GB"),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="number_of_stars",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=1, high=5),
        convert_to="int",  # Convert the sampled float to an integer.
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
    )
)

config_builder.validate()

[19:00:10] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
)

## 🦜 LLM-generated columns

- The real power of Data Designer comes from leveraging LLMs to generate text, code, and structured data.

- When prompting the LLM, we can use Jinja templating to reference other columns in the dataset.

- As we see below, nested json fields can be accessed using dot notation.


In [8]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="product_name",
        prompt=(
            "You are a helpful assistant that generates product names. DO NOT add quotes around the product name.\n\n"
            "Come up with a creative product name for a product in the '{{ product_category }}' category, focusing "
            "on products related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. Respond with only the product name, no other text."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.add_column(
    LLMTextColumnConfig(
        name="customer_review",
        prompt=(
            "You are a customer named {{ customer.first_name }} from {{ customer.city }}, {{ customer.state }}. "
            "You are {{ customer.age }} years old and recently purchased a product called {{ product_name }}. "
            "Write a review of this product, which you gave a rating of {{ number_of_stars }} stars. "
            "The style of the review should be '{{ review_style }}'."
        ),
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
    )
)

config_builder.validate()

[19:00:10] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
    llm_text_columns: ['product_name', 'customer_review']
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [9]:
preview = data_designer.preview(config_builder)

[19:00:10] [INFO] 🧐 Preview generation in progress
[19:00:10] [INFO] ✅ Validation passed
[19:00:10] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[19:00:10] [INFO] 🩺 Running health checks for models...
[19:00:10] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[19:00:13] [INFO]   |-- ✅ Passed!
[19:00:13] [INFO] 🎲 Preparing samplers to generate 10 records across 6 columns
[19:00:13] [INFO] 📝 Preparing llm-text column generation
[19:00:13] [INFO]   |-- column name: 'product_name'
[19:00:13] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
        "max_parallel_requests": 4,
        "timeout": null,
        "extra_body": null
    },
    "provider": "nvidia"
}
[19:00:13] [INFO] 🐙 Processing llm-text column 'product_name' with 4 c

In [10]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category    │ Home & Kitchen                                                                            │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Organization                                                                              │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 50-65                                                                                     │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer            │ {                                                                                         │
│                     │     'uuid': '67c0a53a-ddfb-49c4-adea-69a220eef85f',                                       │
│                     │     'locale': 'en_GB',                                                                    │
│                     │     'first_name': 'Angela',                                                               │
│                     │     'last_name': 'Atkins',                                                                │
│                     │     'middle_name': None,                                                                  │
│                     │     'sex': 'Female',                                                                      │
│                     │     'street_number': '3',                                                                 │
│                     │     'street_name': 'Ashley turnpike',                                                     │
│                     │     'city': 'Mohammedview',                                                               │
│                     │     'state': None,                                                                        │
│                     │     'postcode': 'WV02 7UH',                                                               │
│                     │     'age': 31,                                                                            │
│                     │     'birth_date': '1994-05-17',                                                           │
│                     │     'country': 'Bahamas',                                                                 │
│                     │     'marital_status': 'widowed',                                                          │
│                     │     'education_level': 'doctorate',                                                       │
│                     │     'unit': '',                                                                           │
│                     │     'occupation': 'Metallurgist',                                                         │
│                     │     'phone_number': '0117 496 0534',                                                      │
│                     │     'bachelors_field': 'arts_humanities'                                                  │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ number_of_stars     │ 4                                                                                         │
├─────────────────────┼─────────────────────────────────

In [11]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Home & Kitchen,Organization,50-65,{'uuid': '67c0a53a-ddfb-49c4-adea-69a220eef85f...,4,detailed,CozyNest Organizer\n,**Product Review: CozyNest Organizer (4 Stars)...
1,Books,Textbooks,18-25,{'uuid': 'cb555c14-2773-4b91-9052-d8f736b34c79...,4,brief,SmartStudyGuides\n,**Review:** \nSmartStudyGuides is a solid too...
2,Home & Kitchen,Organization,35-50,{'uuid': 'd140c09d-ff91-4259-86bd-bfa86365f8b0...,4,detailed,SmartSpace Organizer\n,**Product Review: SmartSpace Organizer (4 Star...
3,Home Office,Lighting,25-35,{'uuid': '386fc57c-e769-442c-8fbb-7e92348fe052...,5,structured with bullet points,BrightNest\n,**Trevor's Review of BrightNest (5 Stars) – La...
4,Home & Kitchen,Furniture,25-35,{'uuid': 'a2dd98a8-6e95-4953-9c6d-0e35c40cb976...,3,detailed,ModernNest\n,**Product Review: ModernNest – A 3-Star Experi...
5,Books,Fiction,35-50,{'uuid': '4156a3d2-79d4-4b72-9fdb-4dd22b477935...,3,detailed,Whispers of the Forgotten Realm\n,"**Review Title:** A Mixed Experience with ""Whi..."
6,Electronics,Cameras,65+,{'uuid': '44af7f7c-2aa4-4cba-8099-41b47767c0f8...,3,detailed,SeniorShutter\n,**Product Review: SeniorShutter – A Mixed Expe...
7,Clothing,Men's Clothing,50-65,{'uuid': '91d59484-dba8-494e-a9c1-f5e90044ac2b...,4,rambling,TimelessTactics\n,"Okay, so I bought this thing called TimelessTa..."
8,Home Office,Desks,25-35,{'uuid': '8b5c6ead-a39b-4ddf-bf36-c91af15cfe90...,5,detailed,WorkNest Pro\n,**Product Review: WorkNest Pro – A Game-Change...
9,Books,Self-Help,50-65,{'uuid': '7ba8a0a6-f2a9-4145-b887-64957ee40158...,4,structured with bullet points,Mindful Moments\n,**Product Review: Mindful Moments – 4 Stars** ...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [12]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                          5 (50.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                          9 (90.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (50.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ customer                         │             dict │                        10 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ number_of_stars                  │              int │                          3 (30.0%) │              uniform │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (40.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                10 (100.0%) │      77.0 +/- 0.7 │            4.5 +/- 1.0 │
├───────────────────────┼───────────────┼─────────────────

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [13]:
results = data_designer.create(config_builder, num_records=20)

[19:00:31] [INFO] 🎨 Creating Data Designer dataset
[19:00:31] [INFO] ✅ Validation passed
[19:00:31] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[19:00:31] [INFO] 🩺 Running health checks for models...
[19:00:31] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[19:00:32] [INFO]   |-- ✅ Passed!
[19:00:32] [INFO] ⏳ Processing batch 1 of 1
[19:00:32] [INFO] 🎲 Preparing samplers to generate 20 records across 6 columns
[19:00:32] [INFO] 📝 Preparing llm-text column generation
[19:00:32] [INFO]   |-- column name: 'product_name'
[19:00:32] [INFO]   |-- model config:
{
    "alias": "nemotron-nano-v2",
    "model": "nvidia/nvidia-nemotron-nano-9b-v2",
    "inference_parameters": {
        "temperature": 0.5,
        "top_p": 1.0,
        "max_tokens": 1024,
        "max_parallel_requests": 4,
        "timeout": null,
        "extra_body": null
    },
    "provider": "nvidia"
}
[19:00:32] [INFO] 🐙 Proces

In [14]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,customer_review
0,Electronics,Headphones,35-50,"{'age': 44, 'bachelors_field': 'arts_humanitie...",3,detailed,SoundSage,**Review Title: A Mixed Experience with SoundS...
1,Home Office,Storage,50-65,"{'age': 52, 'bachelors_field': 'no_degree', 'b...",3,structured with bullet points,TimelessTote,**Review of TimelessTote (3 Stars)** - **De...
2,Electronics,Accessories,25-35,"{'age': 56, 'bachelors_field': 'no_degree', 'b...",4,detailed,NeonPulse,**Product Review: NeonPulse by Mathew from Kie...
3,Electronics,Smartphones,65+,"{'age': 48, 'bachelors_field': 'no_degree', 'b...",4,detailed,SeniorSense Smartphone,**Product Review: SeniorSense Smartphone – A T...
4,Home Office,Desks,35-50,"{'age': 48, 'bachelors_field': 'no_degree', 'b...",2,brief,WorkspaceElevate,**Review:** Disappointing. Expected more from ...


In [15]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                          5 (25.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                         14 (70.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (25.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ customer                         │             dict │                        20 (100.0%) │               person │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ number_of_stars                  │              int │                          4 (20.0%) │              uniform │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (20.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                                📝 LLM-Text Columns                                                
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_name          │        string │                20 (100.0%) │      77.0 +/- 0.8 │            4.5 +/- 0.9 │
├───────────────────────┼───────────────┼─────────────────

## ⏭️ Next Steps

Now that you've seen the basics of Data Designer, check out the following notebooks to learn more about:

- [Structured outputs and jinja expressions](/notebooks/2-structured-outputs-and-jinja-expressions/)

- [Seeding synthetic data generation with an external dataset](/notebooks/3-seeding-with-a-dataset/)
